In [1]:
# Imports
import os
import csv
import cv2
import pandas as pd
import numpy as np
import mediapipe as mp
from mediapipe.python.solutions.face_mesh_connections import FACEMESH_CONTOURS

In [2]:
mp_drawing = mp.solutions.drawing_utils 
mp_holistic = mp.solutions.holistic 

In [3]:
# Total landmarks
TOTAL_FACE_LANDMARKS  = 468
TOTAL_RHAND_LANDMARKS = 21
TOTAL_LHAND_LANDMARKS = 21
TOTAL_POSE_LANDMARKS  = 33

In [4]:
# Create Dataframe columns names
## Columns names list
list_column_names = []
## WORD
list_column_names.append("Word")
## FACE
for i in range(TOTAL_FACE_LANDMARKS):
    list_column_names.append("Face-"+str(i))
## RIGHT HAND
for i in range(TOTAL_RHAND_LANDMARKS):
    list_column_names.append("RHand-"+str(i))
## LEFT HAND
for i in range(TOTAL_LHAND_LANDMARKS):
    list_column_names.append("LHand-"+str(i))
## POSE
for i in range(TOTAL_RHAND_LANDMARKS):
    list_column_names.append("Pose-"+str(i))

In [5]:
df = pd.DataFrame(columns = [list_column_names])
df

Empty DataFrame
Columns: [(Word,), (Face-0,), (Face-1,), (Face-2,), (Face-3,), (Face-4,), (Face-5,), (Face-6,), (Face-7,), (Face-8,), (Face-9,), (Face-10,), (Face-11,), (Face-12,), (Face-13,), (Face-14,), (Face-15,), (Face-16,), (Face-17,), (Face-18,), (Face-19,), (Face-20,), (Face-21,), (Face-22,), (Face-23,), (Face-24,), (Face-25,), (Face-26,), (Face-27,), (Face-28,), (Face-29,), (Face-30,), (Face-31,), (Face-32,), (Face-33,), (Face-34,), (Face-35,), (Face-36,), (Face-37,), (Face-38,), (Face-39,), (Face-40,), (Face-41,), (Face-42,), (Face-43,), (Face-44,), (Face-45,), (Face-46,), (Face-47,), (Face-48,), (Face-49,), (Face-50,), (Face-51,), (Face-52,), (Face-53,), (Face-54,), (Face-55,), (Face-56,), (Face-57,), (Face-58,), (Face-59,), (Face-60,), (Face-61,), (Face-62,), (Face-63,), (Face-64,), (Face-65,), (Face-66,), (Face-67,), (Face-68,), (Face-69,), (Face-70,), (Face-71,), (Face-72,), (Face-73,), (Face-74,), (Face-75,), (Face-76,), (Face-77,), (Face-78,), (Face-79,), (Face-80,), (Face-81,), (Face-82,), (Face-83,), (Face-84,), (Face-85,), (Face-86,), (Face-87,), (Face-88,), (Face-89,), (Face-90,), (Face-91,), (Face-92,), (Face-93,), (Face-94,), (Face-95,), (Face-96,), (Face-97,), (Face-98,), ...]
Index: []

[0 rows x 532 columns]

In [12]:
with open('coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(df)

In [17]:
# Read Videos
path = "../abajoprueba"
words = [words for words in os.listdir(path) if os.path.isdir(os.path.join(path, words))]
# All videos must have the same resolution (400x300)
HEIGHT = 400
WIDTH  = 300
for word in words:
    for video in os.listdir(os.path.join(path, word)):
        # Path to each video
        video_path = os.path.join(os.path.join(path, word), video)
        print(video_path)
        # Capture video
        cap = cv2.VideoCapture(video_path)
        # Initialize holistic model
        with mp_holistic.Holistic(min_detection_confidence = 0.5, min_tracking_confidence = 0.5) as holistic:
            while cap.isOpened():
                # Read frame
                ret, frame = cap.read()
                if ret == True:
                    # Resize frame
                    frame = cv2.resize(frame, (WIDTH, HEIGHT), interpolation = cv2.INTER_AREA)
                    # Change color from BGR to RGB
                    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    frame.flags.writeable = False
                    # Detect landmarks
                    results = holistic.process(frame)
                    # Get the coordinates
                    
                    
                    
                    # CARA
                    mp_drawing.draw_landmarks(
                        frame_rgb, results.face_landmarks, mp_holistic.FACEMESH_CONTOURS,
                        mp_drawing.DrawingSpec(color=(0, 255, 255), thickness=1, circle_radius=1),
                        mp_drawing.DrawingSpec(color=(0, 128, 255), thickness=2))
               
                   # Mano izquieda (azul)
                    mp_drawing.draw_landmarks(
                        frame_rgb, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(255, 255, 0), thickness=2, circle_radius=1),
                        mp_drawing.DrawingSpec(color=(255, 0, 0), thickness=2))
               
                   # Mano derecha (verde)
                    mp_drawing.draw_landmarks(
                        frame_rgb, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(0, 255, 0), thickness=2, circle_radius=1),
                        mp_drawing.DrawingSpec(color=(57, 143, 0), thickness=2))
               
                   # Postura
                    mp_drawing.draw_landmarks(
                        frame_rgb, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                        mp_drawing.DrawingSpec(color=(128, 0, 255), thickness=2, circle_radius=1),
                        mp_drawing.DrawingSpec(color=(255, 255, 255), thickness=2))

          
               
                      # Export coordinates
                    try:
                        # Extraer cuerpo (pose_landmarks) 
                        pose = results.pose_landmarks.landmark
                        pose_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in pose]).flatten())
               
                        # Extraer cara (face_landmarks)
                        face = results.face_landmarks.landmark
                        face_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in face]).flatten())
                    
                        try:
                            # Extraer mano derecha landmarks(right_hand_landmarks)
                            handr = results.right_hand_landmarks.landmark
                            handr_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in handr]).flatten())
                    
                        except:
                            pass
               
                        try:
                        #Extraer mano izquierda landmarks(left_hand_landmarks)
                            handl = results.left_hand_landmarks.landmark
                            handl_row = list(np.array([[landmark.x, landmark.y, landmark.z, landmark.visibility] for landmark in handl]).flatten())
                    
                        except:
                            pass
               
               
                        # Concate rows ( UNIR ROWS)
                    
                    
                        try:
                            row = pose_row + face_row + handr_row+handl_row
                        except:                        
                            try:
                                row = pose_row + face_row + handr_row 
                            except:                                          
                                try:
                                    row = pose_row + face_row + handl_row
                                except:
                                    pass
                    
                        # Append class name 
                        row.insert(0, class_name) # insertar nombre en columna 0
                    
                   
                                   
               
                    except:
                        pass
                   #frame = cv2.flip(frame,1) #para voltear la imagen en caso de selfie
                    cv2.imshow("Imagen a detectar", frame)
          
                    if (cv2.waitKey(1) & 0xFF == ('s')):
                        break                

                    # Export to CSV
                    with open('coords.csv', mode='a', newline='') as f:
                       csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                       csv_writer.writerow(df) 

                    
                    
               
                    
                else:
                    break
            cap.release()
        break # Este break es para que solo coja el primer video y asi hacer pruebas

FileNotFoundError: [WinError 3] El sistema no puede encontrar la ruta especificada: '../abajoprueba'

In [21]:
results

mediapipe.python.solution_base.SolutionOutputs